# Welcome to Parallel Compute Hackathon
---
This notebook uses [pyTigerGraph](https://pytigergraph.github.io/pyTigerGraph/), a TigerGraph python interface to calculate similarity using Xilinx FPGAs. The python package uses a Java client to run gsql queries on a remote server running TigerGraph via Rest APIs.

#### The demo
For the purpose of this demo, we will use medical data synthetically generated using an open-source tool called [Synthea](https://synthetichealth.github.io/synthea/) to find patients that are the most "similar" to a particular patient based on a list of attributes. The similarity score used is Cosine Similarity. For more information, refer to [Xilinx Alveo Graph Analytics](https://pages.gitenterprise.xilinx.com/FaaSApps/graphanalytics/index.html) doc. The demo is structured in two sections. Before proceeding, we configure the demo with the following setup steps.
 
> **NOTE 1**: The synthea data is for demonstration purpose only and the user can use their own data and write queries to create their own embeddings.

> **NOTE 2**: The TigerGraph plugin currently used to run queries on FPGA supports only one graph. Therefore, more than one concurrent users running this demo on the same TG server is not supported. This will be supported in a future version.

### Setup
---
Boilerplate module imports

In [ ]:
import time
import random as rand
from pathlib import Path, PurePosixPath
import pyTigerGraph as tg

#### Login Setup
Provide the remote TigerGraph server URL/IP address/hostname and credentials for a TigerGraph user. 

**NOTE**: The TigerGraph user should be created on the server side before proceeding

In [ ]:
hostName = "localhost"                            # TG server hostname
userName = "dummy"                                # TG user name
passWord = "password"                             # TG user password

#### Demo variables
Set variables to specify scale of the demo. Use these to: 
- set larger population sizes - the corresponding data for the population size should exist on the remote server
- set number of best matching patients to return
- set number of FPGAs to run the query on - larger number leads to larger acceleration but the TigerGraph server should have that many FPGA cards installed

In [ ]:
populationSize = 1000                               # Size of the total patient population
topK = 10                                           # Number of highest scoring patient matches
numDevices = 1                                      # Number of FPGA devices to distribute the queries to

#### Path Setup
**Local**: Location of query files under the Xilinx graphanalytics github repo. Set location of the local repo.

In [ ]:
localRepoLocation = Path("C:/Users/dummy")
exampleLocation = Path("graphanalytics/plugin/tigergraph/examples/synthea") # when running from github repo
queryFileLocation = localRepoLocation / exampleLocation / "query"

**Remote**: Location of synthea generated data on the server. **NOTE**: Data should exist on the TigerGraph server 

In [ ]:
serverRepoLocation = PurePosixPath("/home/dummy")
serverDataLocation = serverRepoLocation / PurePosixPath(exampleLocation) / "1000_patients/csv"

#### Utility Methods

In [ ]:
def getPatient(id):
    patientList = conn.getVerticesById('patients', id)
    return [] if len(patientList) == 0 else patientList[0]

def getPatientName(patient):
    return patient['attributes']['FIRST_NAME'] + ' ' + patient['attributes']['LAST_NAME']

def printResults(result, newPatient):
    matches = result[0]['Matches']
    print(f'Matches for patient {getPatientName(newPatient)}')
    for m in matches:
        matchingPatient = getPatient(m['Id'])
        print(f'{m["score"]} {getPatientName(matchingPatient)}')

---
#### Prepare TG database
Shows **one-time** preparation of the database. Once done, queries can be repeateadly run as shown in the next Section.
1. [**Load Graph**](#loadg)
 - [Create new graph](#newg)
 - [Create graph schema](#schema)
 - [Load graph data](#loadd)
 - [Install queries](#install)


2. [**Create Embeddings**](#embed)
3. [**Send Embeddings to FPGA**](#send)

#### Run Queries on FPGA
Shows **repeatable** use of query to run *accelerated* similarity computation on FPGA
1. [**Compute Cosine Similarity**](#run)

The cells below show how to perform these steps in detail.

### 1. Load Graph <a id="loadg"></a>
---
#### 1.1 Create new graph <a id="newg"></a>
- Connect to TigerGraph server by ommiting graph name. This is needed to establish a valid REST endpoint that will be used to create a new desired graph
- Create new graph by using gsql command and create a new connection with the new graph

In [ ]:
# connect to TG server and create graph
graphName = f'xgraph_{userName}_{populationSize}'   # TG graph name
conn = tg.TigerGraphConnection(host='http://' + hostName, graphname='', username=userName, password=passWord, useCert=False)
print("\n--------- Creating New graph ----------")
print(conn.gsql(f'create graph {graphName}()', options=[]))

# connect to TG server with new graph
print(f'Using graph {graphName}')
conn = tg.TigerGraphConnection(host='http://' + hostName, graphname=graphName, username=userName, password=passWord, useCert=False)

Any command or query will now run on the new graph.

#### 1.2 Create graph schema <a id="schema"></a>
TigerGraph stores graph in the form of vertices that can be associated with other vertices using directed or undirected edges. This is specified in the form of a graph schema. For the purpose of this demo, the schema is already defined as a query file. Load the file, set graph name and run it as gsql commands. 

The user can create schema for their own graph in a similar way. 

In [ ]:
print("\n--------- Creating New Schema ----------")
schemaFile = queryFileLocation / "schema_xgraph.gsql"

with open(schemaFile) as fh:
    qStrRaw = fh.read()
    qStr = qStrRaw.replace('@graph', graphName)
    print(conn.gsql(qStr, options=[]))

#### 1.3 Load graph data <a id="loadd"></a>
The synthea data is split into files for each patient attribute (vertex in the schema). Each file is loaded and parsed. Open the load query file and, set graph name and location of the data files.

In [ ]:
print("\n--------- Loading data into graph ----------")
loadFile = queryFileLocation / "load_xgraph.gsql"
print(str(serverDataLocation))

with open(loadFile) as fh:
    qStrRaw = fh.read()
    qStrRaw = qStrRaw.replace('@graph', graphName)
    qStr    = qStrRaw.replace('$sys.data_root', str(serverDataLocation))
    print(conn.gsql(qStr, options=[]))
    print(conn.gsql(f"RUN LOADING JOB load_xgraph", options=['-g', graphName]))
    print(conn.gsql(f"DROP JOB load_xgraph", options=['-g', graphName]))

#### 1.4 Install queries <a id="install"></a>
The cosine similarity application functionality is implemented using gsql queries and UDF functions. The queries need to be installed before running.

The user can create their own queries and install them instead. If user writes their own UDFs, they will need to be compilled and opened as a TigerGraph Plugin (this is not covered in the scope of this demo).

In [ ]:
print("\n--------- Installing Queries ----------")
baseQFile = queryFileLocation / "base.gsql"
clientQFile = queryFileLocation / "client.gsql"

with open(baseQFile) as bfh, open(clientQFile) as cfh:
    print("installing base queries ...")
    qStrRaw = bfh.read()
    qStr = qStrRaw.replace('@graph', graphName)
    print(conn.gsql(qStr, options=[]))
    
    print("\ninstalling client queries ...")
    qStrRaw = cfh.read()
    qStr = qStrRaw.replace('@graph', graphName)
    print(conn.gsql(qStr, options=[]))

Now that queries are installed, rest of the operations can be performed simply by running the queries as follows.

### 2. Create Embeddings <a id="embed"></a>
---
As seen earlier in the schema, each patient has a set of attributes which are represented as vertices. Additionaly, each patient is also represented as a vertex. The attributes of a patient are embedded into a vector representation called embeddings which are then stored as part of the patient vertex. Refer to [Cosine similarity](https://pages.gitenterprise.xilinx.com/FaaSApps/graphanalytics/overview.html#cosine-similarity) section for details on how the attributes are mapped to the patient embedding vector.

In [ ]:
print('Creating patient embeddings and storing them in patient vertices...')
tStart = time.perf_counter()
conn.runInstalledQuery('client_cosinesim_embed_vectors', timeout=240000000)
conn.runInstalledQuery('client_cosinesim_embed_normals', timeout=240000000)
print(f'completed in {time.perf_counter() - tStart:.4f} sec')

### 3. Send embeddings to FPGA <a id="send"></a>
---
Finally, the embeddings are collected in a buffer which is sent/copied to HBM memory on the FPGA device. 

In [ ]:
print('Loading data into FPGA memory...')
# set number of FPGAs to use
conn.runInstalledQuery('client_cosinesim_set_num_devices', {'numDevices': numDevices}, timeout=240000000)

# distribute data to FPGA memory
tStart = time.perf_counter()
resultHwLoad = conn.runInstalledQuery('client_cosinesim_load_alveo', timeout=240000000)
print(f'completed in {time.perf_counter() - tStart:.4f} sec\n')

# Check status
status = conn.runInstalledQuery('client_cosinesim_get_alveo_status', timeout=240000000)
isInit = status[0]["IsInitialized"]
numDev = status[0]["NumDevices"]
print(f'FPGA Init: {isInit}, Dev: {numDev}\n')

This completes the TigerGraph database and consine similiarity compute preparation. We can now run as many similarity queries as we want. 

### Compute Cosine Similarity <a id="run"></a>
---
For the purpose of this demo, we get the first 100 patients and choose one at random. Patients are represented by an ID which is passed to the match query.

In [ ]:
print('Running Query...')
# pick a random patient out of 100
targetPatients = conn.getVertices('patients', limit=100)
targetPatient = targetPatients[rand.randint(1,100)]

# run similarity on the choosen patient
tStart = time.perf_counter()
result = conn.runInstalledQuery('client_cosinesim_match_alveo',
                                  {'newPatient': targetPatient['v_id'], 'topK': topK}, timeout=240000000)
tDuration = 1000*(time.perf_counter() - tStart)

printResults(result, targetPatient)
resTime = result[0]["ExecTimeInMs"]
print(f"\nRound Trip time: {tDuration:.2f} msec")
print(f"     Query time: {resTime:.2f} msec")

Notice that as a sanity check, the top matching patient is the query patient itself with a match score of 1.
Feel free to play with the query!

#### Thanks for your time!